In [ ]:
!python -c 'print(__import__("sys").version[:3])'

3.7


Требуется обновить SQLite, чтобы можно было использовать оконные функции.

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz
!tar xzf sqlite.tar.gz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.$(python -V | cut -d"." -f2)/pysqlite3/_sqlite3.cpython-3$(python -V | cut -d"." -f2)m-x86_64-linux-gnu.so \
    /usr/lib/python3.$(python -V | cut -d"." -f2)/lib-dynload/_sqlite3.cpython-3$(python -V | cut -d"." -f2)m-x86_64-linux-gnu.so

--2022-05-08 22:47:08--  https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release
Resolving www.sqlite.org (www.sqlite.org)... 45.33.6.223, 2600:3c00::f03c:91ff:fe96:b959
Connecting to www.sqlite.org (www.sqlite.org)|45.33.6.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11673633 (11M) [application/x-compressed]
Saving to: ‘sqlite.tar.gz’

sqlite.tar.gz       100%[===================>]  11.13M  57.9MB/s    in 0.2s    

2022-05-08 22:47:09 (57.9 MB/s) - ‘sqlite.tar.gz’ saved [11673633/11673633]

/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
chec

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
sqlite3.sqlite_version

'3.38.5'

Есть таблица с данными по продажам кофе:

* `SALE_DTTM` – точное время продажи,
* `CARD_NUMBER` – номер скидочной карты
* `COFFEE_TYPE` – название проданного кофейного напитка
* `PRICE` – полная стоимость
* `DISCOUNT` – размер скидки в %
* `GROCERY_FLG` – флаг покупки выпечки в том же чеке

Составьте алгоритм SQL-запросов для решения следующих заданий:

- Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц (при отсутствии встроенной функции, вычисляющей медиану).

- Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

- В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой. При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе 
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с выпечкой. Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки клиента
 не является константой).

## Подготавливаем исходные данные

In [3]:
dt = pd.DataFrame({'date': pd.date_range('2022-02-01', '2022-03-31')})

In [4]:
rs = np.random.RandomState(42)

In [5]:
dt['num'] = rs.poisson(30, size=len(dt))

In [6]:
df = pd.DataFrame().assign(
    SALE_DTTM=dt['date'].apply(lambda x: [x]).mul(dt['num']).sum()
)
df = df.assign(
    CARD_NUMBER=pd.Series(list(range(50))).sample(
        len(df),
        replace=True,
        random_state=42,
        ignore_index=True
    ),
    COFFEE_TYPE=rs.choice(
        ['espresso', 'americano','cappuccino', 'latte'],
        size=len(df),
        p=[1/6,1/6,1/3,1/3] #rs.dirichlet(np.ones(4), size=1).flatten()
    )
)

In [7]:
coffee_price = pd.DataFrame({'COFFEE_TYPE': df['COFFEE_TYPE'].unique(),
                             'COFFEE_PRICE': range(100, 160, 15)})

In [ ]:
coffee_price

,COFFEE_TYPE,COFFEE_PRICE
0,espresso,100
1,americano,115
2,latte,130
3,cappuccino,145


In [8]:
df = df.merge(coffee_price, how='left', on='COFFEE_TYPE').assign(
    DISCOUNT=rs.choice([.0,.05,.1,.15], p=[.4,.3,.2,.1], size=len(df)),
    GROCERY_FLG=rs.choice([0,1], p=[2/3, 1/3], size=len(df))
)

In [9]:
df.loc[df['CARD_NUMBER'].sample(10, replace=False, random_state=42).index,
       ['CARD_NUMBER', 'DISCOUNT']] = np.nan, 0

In [ ]:
df[df.CARD_NUMBER.isna()]

,SALE_DTTM,CARD_NUMBER,COFFEE_TYPE,COFFEE_PRICE,DISCOUNT,GROCERY_FLG
212,2022-02-08,NaN,cappuccino,145,0.0,0
450,2022-02-16,NaN,americano,115,0.0,1
529,2022-02-19,NaN,cappuccino,145,0.0,0
765,2022-02-28,NaN,cappuccino,145,0.0,1
1107,2022-03-10,NaN,cappuccino,145,0.0,0
1174,2022-03-13,NaN,latte,130,0.0,1
1296,2022-03-17,NaN,latte,130,0.0,0
1518,2022-03-24,NaN,americano,115,0.0,0
1701,2022-03-29,NaN,cappuccino,145,0.0,0
1725,2022-03-30,NaN,latte,130,0.0,1


In [ ]:
df['GROCERY_FLG'].mean()

0.3219954648526077

In [10]:
df['PRICE'] = (df['COFFEE_PRICE'] + df['GROCERY_FLG'] * 100) * (1 - df['DISCOUNT'])

In [11]:
con = sqlite3.connect('db')

In [12]:
cur = con.cursor()

In [13]:
df.to_sql('coffee_sales', con, index=False, if_exists='replace')

In [ ]:
query = """
SELECT * FROM coffee_sales t LIMIT 10
"""

In [14]:
def select(query):
  return pd.read_sql(query, con)

In [ ]:
select(query)

,SALE_DTTM,CARD_NUMBER,COFFEE_TYPE,COFFEE_PRICE,DISCOUNT,GROCERY_FLG,PRICE
0,2022-02-01 00:00:00,38.0,espresso,100,0.15,0,85.00
1,2022-02-01 00:00:00,28.0,americano,115,0.00,0,115.00
2,2022-02-01 00:00:00,14.0,latte,130,0.05,1,218.50
3,2022-02-01 00:00:00,42.0,americano,115,0.10,0,103.50
4,2022-02-01 00:00:00,7.0,espresso,100,0.10,1,180.00
5,2022-02-01 00:00:00,20.0,cappuccino,145,0.05,0,137.75
6,2022-02-01 00:00:00,38.0,latte,130,0.00,1,230.00
7,2022-02-01 00:00:00,18.0,americano,115,0.05,0,109.25
8,2022-02-01 00:00:00,22.0,latte,130,0.05,1,218.50
9,2022-02-01 00:00:00,10.0,latte,130,0.00,0,130.00


## 1. Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц (при отсутствии встроенной функции, вычисляющей медиану).

In [ ]:
query = """
SELECT
  month
  , AVG(price_sum) AS mean_price
  --, MEDIAN(price_sum) AS median_price
FROM (
  SELECT
    DATE(SALE_DTTM, 'start of month') AS month
    , card_number
    , SUM(PRICE) AS price_sum
  FROM coffee_sales
  GROUP BY
    DATE(SALE_DTTM, 'start of month')
    , CARD_NUMBER) cs
GROUP BY 1
"""

In [ ]:
select(query)

,month,mean_price
0,2022-02-01,2949.579268
1,2022-03-01,3654.670732


Функция расчета медианы на python

In [15]:
def custom_median(x):
   sorted_x = sorted(x)
   len_x = len(x)
   len_x_half = len_x // 2
   return sorted_x[len_x_half] if len_x % 2 \
          else sum(sorted_x[len_x_half - 1 * len_x % 2: len_x_half + 1]) / 2

In [16]:
df.groupby([df['SALE_DTTM'].dt.month, 'CARD_NUMBER'])['PRICE'] \
  .sum() \
  .reset_index() \
  .groupby('SALE_DTTM')['PRICE'] \
  .apply(custom_median)

SALE_DTTM
2    2305.75
3    2984.00
Name: PRICE, dtype: float64

https://www.sqlservercentral.com/forums/topic/calculating-a-median-for-groups

```sql
PERCENTILE_CONT(col, 0.5) OVER(PARTITION BY col1) AS median
```

В SQL медиану можно рассчитать, используя аналитические оконные функции `ROW_NUMBER()` и `COUNT()`. Используем эти для вычислния номеров и количества строк в окне. Дальше воспользуемя тем, что при делении на целое число в SQL получаем целочисленный ответ.
Выберем из кажого окна строки с номерами, получившимися при делении суммарного количества строк плюс 1 и плюс 2 на 2, возьмем среднее от этих значений (AVG нам использовать можно).

|num_row|value|
|--|--|
|1|10|
|2|20|
|**3**|**30**|
|4|40|
|5|50|

Количество строк — 5, (5+1)/2 = 3, (5+2)/2 = 3. Оба значения равны 3. Значение в строках под номером 3 — 30. Среднее (30+30)/2 = 30.0 — это и есть искомая медиана.

|num_row|value|
|--|--|
|1|10|
|2|20|
|**3**|**30**|
|**4**|**40**|
|5|50|
|6|60|

Количество строк — 6, (6+1)/2 = 3, (6+2)/2 = 4, Значения в строках 3 и 4 — 30 и 40 соответственно. Среднее (30+40)/2 = 35.0 — это и есть искомая медиана.


In [17]:
query = """
WITH s AS (
  SELECT
    DATE(SALE_DTTM, 'start of month') AS month
    , CARD_NUMBER 
    , SUM(PRICE) AS price_sum
  FROM coffee_sales
  WHERE CARD_NUMBER IS NOT NULL
  GROUP BY 1, 2
),
r AS (
  SELECT 
    *
    , ROW_NUMBER() OVER (PARTITION BY month ORDER BY price_sum) AS rn
    , COUNT(*) OVER (PARTITION BY month) AS rc
  FROM s 
)

SELECT 
  month
  , AVG(price_sum) AS median
FROM r
WHERE rn IN ((rc + 1) / 2, (rc + 2) / 2)
GROUP BY month
"""

In [18]:
select(query)

,month,median
0,2022-02-01,2305.75
1,2022-03-01,2984.00


# 2. Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.


По каждому клиенту выбираются дни, в которых вместе с кофе покупалась любая выпечка. Затем по этим данным строятся сессии, в которых такие покупки шли одна за другой без перерыва, то есть каждый день. Для этого мы сравниваем дату текущего дня минус один день со смещенной датой (LAG), что позволит при коммулятивном суммировании сформировать группы непрерывных сессий.

In [19]:
sales_by_day = df.groupby([df['SALE_DTTM'].dt.date, 'CARD_NUMBER'])['GROCERY_FLG'] \
                 .apply(max).reset_index()

In [21]:
def longest_session(df):
    df = df.assign(SESSION=1)
    df.loc[(df['SALE_DTTM'] - pd.Timedelta(days=1) == df['SALE_DTTM'].shift()) 
            & (df['GROCERY_FLG'] == 1), 'SESSION'] = 0
    df['SESSION'] = df['SESSION'].cumsum()
    
    return df

In [22]:
sales_by_day.groupby('CARD_NUMBER')[['SALE_DTTM', 'GROCERY_FLG']] \
            .apply(longest_session).reset_index(level=1, drop=True) \
            .groupby(['CARD_NUMBER', 'SESSION'])['GROCERY_FLG'].sum() \
            .nlargest(10)

CARD_NUMBER  SESSION
0.0          3          3
2.0          17         3
9.0          2          3
10.0         4          3
15.0         5          3
17.0         20         3
18.0         6          3
33.0         10         3
34.0         19         3
43.0         12         3
Name: GROCERY_FLG, dtype: int64

In [23]:
query = """
WITH sales_by_day_with_grocery AS (
  SELECT
    DATE(SALE_DTTM) AS SALE_DTTM
    , CARD_NUMBER
    , MAX(GROCERY_FLG) AS GROCERY_FLG
  FROM coffee_sales
  WHERE CARD_NUMBER IS NOT NULL
  --AND GROCERY_FLG = 1
  GROUP BY 1, 2
  HAVING GROCERY_FLG = 1
),
sale_sessions AS (
  SELECT
    SALE_DTTM,
    CARD_NUMBER ,
    GROCERY_FLG,
    CASE WHEN DATE(SALE_DTTM, '-1 day') = LAG(SALE_DTTM)
      OVER(PARTITION BY CARD_NUMBER ORDER BY SALE_DTTM)
    THEN 0 ELSE 1 END AS session_flag
  FROM sales_by_day_with_grocery
),
sessions_by_card AS (
  SELECT 
    CARD_NUMBER
    , SALE_DTTM
    , GROCERY_FLG
    , SUM(session_flag) OVER(PARTITION BY CARD_NUMBER
      ORDER BY SALE_DTTM ROWS BETWEEN UNBOUNDED PRECEDING
      AND CURRENT ROW) AS session
  FROM sale_sessions
)

SELECT
  CARD_NUMBER
  , COUNT(GROCERY_FLG) AS SESSION_LENGHT
FROM sessions_by_card
GROUP BY CARD_NUMBER, session
ORDER BY 2 DESC
"""

In [24]:
select(query).nlargest(10, columns='SESSION_LENGHT')

,CARD_NUMBER,SESSION_LENGHT
0,0.0,3
1,2.0,3
2,9.0,3
3,10.0,3
4,15.0,3
5,17.0,3
6,18.0,3
7,33.0,3
8,34.0,3
9,43.0,3


## 2. Расчет скидки за серию покупок

Скидка расчитывается в границах сессии непрерывных (каждый день) покупок кофе с выпечкой по формуле `1-(1-discounts).prod()`, где discounts — массив скидок внутри каждой сессии. Чем длиннее череда покупок и больше размер скидок внутри сессии, тем существеннее финальная скидка.

In [25]:
df.query('GROCERY_FLG == 1') \
  .groupby([df['SALE_DTTM'].dt.date,
            'CARD_NUMBER'])[['GROCERY_FLG', 'DISCOUNT']].max().reset_index() \
  .groupby('CARD_NUMBER')[['SALE_DTTM', 'GROCERY_FLG', 'DISCOUNT']] \
            .apply(longest_timeline).reset_index(level=1, drop=True) \
            .groupby(['CARD_NUMBER', 'SESSION']) \
            .agg(SESSION_LENGHT=('GROCERY_FLG', 'count'), 
                                                      #1-np.exp(np.sum(np.log(1-x)))
                 TOTAL_DISCOUNT=('DISCOUNT', lambda x: 1-(1-x).prod())) \
            .sort_values('TOTAL_DISCOUNT', ascending=False)

SESSION_LENGHT  TOTAL_DISCOUNT
CARD_NUMBER SESSION                                
9.0         2                     3        0.313625
22.0        12                    2        0.277500
41.0        3                     2        0.277500
10.0        2                     3        0.273250
33.0        5                     3        0.271000
...                             ...             ...
27.0        6                     1        0.000000
28.0        5                     1        0.000000
            6                     1        0.000000
            8                     1        0.000000
49.0        7                     1        0.000000

[431 rows x 2 columns]

https://stackoverflow.com/a/5416210

Реализация `np.prod()` в SQL:

```sql
SELECT EXP(SUM(LOG(Column))) AS Product FROM Table
```

**Создание собственных функций в SQLite3**

```python
con.create_function("LOG", 1, np.log)
con.create_function("EXP", 1, np.exp)
```

## 3. Формирование датафрейма из нескольких файлов с данными

Имеется следующая структура данных - данные записываются для каждого пользователя, совершавшего покупки, каждый день. Под каждую дату есть своя папка, внутри неё папки для каждого пользователя. Внутри каждой папки есть файл `data.csv`, где и хранятся данные. Требуется собрать данные из папки `data` в один датафрэйм, где будут две колонки из самих файлов (`product_id`, `quantity`), а также имя пользователя, и дата этих покупок (соответствует названию папки, где лежит папка с пользователем)

In [181]:
!ls

build	       LICENSE	    README.md	 setup.py   sqlite3.h	   test
build-scripts  MANIFEST.in  sample_data  sqlite     sqlite.tar.gz  TODO
db	       pysqlite3    setup.cfg	 sqlite3.c  src


In [182]:
!pwd

/content


In [26]:
!mkdir data

In [29]:
data = pd.DataFrame({'user_id': rs.choice(range(1, 4), size=10),
                     'dt': pd.date_range('2022-01-24', periods=10, freq='12h'),
                     'product_id': range(1, 11),
                     'quantity': rs.poisson(3, size=10)})
data['dt'] = np.datetime_as_string(data['dt'], unit='D')

In [30]:
data

,user_id,dt,product_id,quantity
0,1,2022-01-24,1,3
1,1,2022-01-24,2,3
2,1,2022-01-25,3,3
3,2,2022-01-25,4,1
4,1,2022-01-26,5,3
5,1,2022-01-26,6,1
6,2,2022-01-27,7,4
7,1,2022-01-27,8,0
8,3,2022-01-28,9,4
9,3,2022-01-28,10,3


In [31]:
import os

In [37]:
user_dir = f"/content/data/"
os.path.join(user_dir, 'data.csv')

'/content/data/data.csv'

In [40]:
for dt in data['dt'].unique():
  date_dir = f'/content/data/{dt}'
  os.makedirs(date_dir, exist_ok=True)

  for user_id in data[data['dt'] == dt]['user_id'].unique():
    user_dir = f"/content/data/{dt}/{user_id}"
    os.makedirs(user_dir, exist_ok=True)
    (data[(data['dt'] == dt) & (data['user_id'] == user_id)][['product_id','quantity']]
     .to_csv(os.path.join(user_dir, 'data.csv')))

In [41]:
import glob
glob.glob('/content/data/*/*/*.csv')

['/content/data/2022-01-24/1/data.csv',
 '/content/data/2022-01-27/1/data.csv',
 '/content/data/2022-01-27/2/data.csv',
 '/content/data/2022-01-26/1/data.csv',
 '/content/data/2022-01-28/3/data.csv',
 '/content/data/2022-01-25/1/data.csv',
 '/content/data/2022-01-25/2/data.csv']

In [42]:
dfs = []
for file_path in glob.glob('/content/data/*/*/*.csv'):
      df = pd.read_csv(file_path, index_col=0)
      dt, uid = file_path.rsplit('/', maxsplit=3)[1:3]
      df['dt'] = dt
      df['user_id'] = int(uid)
      dfs.append(df)

In [43]:
pd.concat(dfs)[data.columns].sort_index().equals(data)

True